In [25]:
import pandas as pd

# Load the Excel file
excel_file = r"C:\Users\shres\Downloads\ADP Bartrack.xlsx"

# Read each worksheet into separate DataFrames, skipping initial blank rows
df1 = pd.read_excel(excel_file, sheet_name='ADP 2023 NY', skiprows=lambda x: x == 1)
df2 = pd.read_excel(excel_file, sheet_name='ADP 2023 NJ', skiprows=lambda x: x == 1)
df3 = pd.read_excel(excel_file, sheet_name='ADP 2023 CT', skiprows=lambda x: x == 1)

# Remove "Oxford Health Plans" and unnamed headers
df1.columns = [col if 'Oxford Health Plans' not in col and 'Unnamed' not in col else '' for col in df1.columns]
df2.columns = [col if 'Oxford Health Plans' not in col and 'Unnamed' not in col else '' for col in df2.columns]
df3.columns = [col if 'Oxford Health Plans' not in col and 'Unnamed' not in col else '' for col in df3.columns]

# Remove the first two columns if they exist in df2 and df3
df2 = df2.iloc[:, 2:] if len(df2.columns) > 2 else df2
df3 = df3.iloc[:, 2:] if len(df3.columns) > 2 else df3

# Replace NaN values with "N/A"
df1 = df1.fillna("N/A")
df2 = df2.fillna("N/A")
df3 = df3.fillna("N/A")

# Concatenate DataFrames horizontally
merged_df = pd.concat([df1, df2, df3], axis=1)

# Convert percentage values to string
def convert_percentage_to_string(value):
    if isinstance(value, float):
        return f"{value * 100:.0f}%"
    elif isinstance(value, str) and value.endswith('%'):
        return value
    return value

# Apply conversion function to all cells
merged_df = merged_df.applymap(convert_percentage_to_string)

# Save merged DataFrame to a new Excel file
merged_df.to_excel('merged_adp_plans10.xlsx', index=False)
merged_df


,,,,,,,,,,,...,,,,,,,,,,
0,New York,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,...,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A
1,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,...,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A
2,Plan Type,N/A,Non Standard,Non Standard,Non Standard,Non Standard,Non Standard,Non Standard,Non Standard,Non Standard,...,Non Standard,Non Standard,Non Standard,Non Standard,Non Standard,Non Standard,Non Standard,Non Standard,Non Standard,Non Standard
3,2022 Tracking Number,N/A,Ox_NYLG_Direct_2021_153 as base build plan,Ox_NYLG_Direct_2021_175 as base build plan,Ox_NYLG_Direct_2021_193 as base build plan,Ox_NYLG_Direct_2021_135 as base build plan,Ox_NYLG_Direct_2021_153 as base build plan,Ox_NYLG_Direct_2021_175 as base build plan,Ox_NYLG_Direct_2021_199 as base build plan,Ox_NYLG_Direct_2021_001 as base build plan,...,Ox_CTLG_Direct_2021_153 as base build plan,Ox_CTLG_Direct_2021_175 as base build plan,Ox_CTLG_Direct_2021_193 as base build plan,Ox_CTLG_Direct_2021_135 as base build plan,Ox_CTLG_Direct_2021_153 as base build plan,Ox_CTLG_Direct_2021_175 as base build plan,Ox_CTLG_Direct_2021_199 as base build plan,Ox_CTLG_Direct_2021_001 as base build plan,Ox_CTLG_Direct_2021_096 as base build plan,Ox_CTLG_Direct_2021_016 as base build plan
4,2022 Cirrus Benefit Plan ID,N/A,MNS0000290,MNS0000291,MNS0000291,MNS0000292,MNS0000293,MNS0000294,MNS0000295,MNS0000296,...,MNS0000290,MNS0000291,MNS0000291,MNS0000292,MNS0000293,MNS0000294,MNS0000295,MNS0000296,MNS0000297,MNS0000298
5,2022 Cirrus Benefit Bundle Option ID,N/A,5605086/234699026,5605686/2346993876,523756/2335787926,5605086/234699026,5832466/246756956,5605086/234699026,554739/2323467836,523456/123456756,...,5605086/234699026,5605686/2346993876,523756/2335787926,5605086/234699026,5832466/246756956,5605086/234699026,554739/2323467836,523456/123456756,4567823/469902623,56235386/234696901
6,2022 Rx Code,N/A,No-Standard Rx,No-Standard Rx,No-Standard Rx,No-Standard Rx,RX66-100,RX66-100,RX66-100,RX38,...,No-Standard Rx,No-Standard Rx,No-Standard Rx,No-Standard Rx,RX66-100,RX66-100,RX66-100,RX38,RX66-100,RX39-100
7,2023 Comments,N/A,No changes except for row 36,No changes except for row 36,No changes except for row 36,No changes except for row 36,No changes except for row 36,No changes except for row 36,No changes except for row 36,No changes except for row 36,...,No changes except for row 36,No changes except for row 36,No changes except for row 36,No changes except for row 36,No changes except for row 36,No changes except for row 36,No changes except for row 36,No changes except for row 36,No changes except for row 36,No changes except for row 36
8,Plan Type,N/A,Non Standard,Non Standard,Non Standard,Non Standard,Non Standard,Non Standard,Non Standard,Non Standard,...,Non Standard,Non Standard,Non Standard,Non Standard,Non Standard,Non Standard,Non Standard,Non Standard,Non Standard,Non Standard
9,2023 Tracking Number,N/A,Ox_NYLG_Direct_2021_153 as base build plan,Ox_NYLG_Direct_2021_175 as base build plan,Ox_NYLG_Direct_2021_193 as base build plan,Ox_NYLG_Direct_2021_135 as base build plan,Ox_NYLG_Direct_2021_153 as base build plan,Ox_NYLG_Direct_2021_175 as base build plan,Ox_NYLG_Direct_2021_199 as base build plan,Ox_NYLG_Direct_2021_001 as base build plan,...,Ox_CYLG_Direct_2021_153 as base build plan,Ox_CTLG_Direct_2021_175 as base build plan,Ox_CTLG_Direct_2021_193 as base build plan,Ox_CTLG_Direct_2021_135 as base build plan,Ox_CTLG_Direct_2021_153 as base build plan,Ox_CTLG_Direct_2021_175 as base build plan,Ox_CTLG_Direct_2021_199 as base build plan,Ox_CTLG_Direct_2021_001 as base build plan,Ox_CTLG_Direct_2021_096 as base build plan,Ox_CTLG_Direct_2021_016 as base build plan


In [26]:
# Delete the first row of the transposed DataFrame
merged_df = merged_df.iloc[2:]

In [27]:
# Replace "N/A" with NaN
merged_df.replace("N/A", pd.NA, inplace=True)

# Remove rows where all values are NaN
merged_df.dropna(how='all', inplace=True)

# Reset index if needed
merged_df.reset_index(drop=True, inplace=True)


C:\Users\shres\AppData\Local\Temp\ipykernel_19256\1687488255.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_df.replace("N/A", pd.NA, inplace=True)
C:\Users\shres\AppData\Local\Temp\ipykernel_19256\1687488255.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_df.dropna(how='all', inplace=True)


In [28]:
# Forward fill "<N/A>" values in the first two columns
merged_df.iloc[:, 0] = merged_df.iloc[:, 0].replace("<N/A>", pd.NA).fillna(method='ffill')
merged_df.iloc[:, 1] = merged_df.iloc[:, 1].replace("<N/A>", pd.NA).fillna(method='ffill')

In [29]:
# Strip extra whitespaces and replace "In Network" and "In-Network" with "IN", and "Out-of-Network" with "OON"
merged_df = merged_df.applymap(lambda x: str(x).strip())

# Replace both "In Network" and "In-Network" within the cells with "IN", and "Out-of-Network" with "OON"
merged_df = merged_df.replace({"In Network": "IN", "In-Network": "IN", "Out-of-Network": "OON"}, regex=True)
# Replace "nan" with empty string
merged_df = merged_df.replace("<NA>", "")
merged_df

,,,,,,,,,,,...,,,,,,,,,,
0,Plan Type,,Non Standard,Non Standard,Non Standard,Non Standard,Non Standard,Non Standard,Non Standard,Non Standard,...,Non Standard,Non Standard,Non Standard,Non Standard,Non Standard,Non Standard,Non Standard,Non Standard,Non Standard,Non Standard
1,2022 Tracking Number,,Ox_NYLG_Direct_2021_153 as base build plan,Ox_NYLG_Direct_2021_175 as base build plan,Ox_NYLG_Direct_2021_193 as base build plan,Ox_NYLG_Direct_2021_135 as base build plan,Ox_NYLG_Direct_2021_153 as base build plan,Ox_NYLG_Direct_2021_175 as base build plan,Ox_NYLG_Direct_2021_199 as base build plan,Ox_NYLG_Direct_2021_001 as base build plan,...,Ox_CTLG_Direct_2021_153 as base build plan,Ox_CTLG_Direct_2021_175 as base build plan,Ox_CTLG_Direct_2021_193 as base build plan,Ox_CTLG_Direct_2021_135 as base build plan,Ox_CTLG_Direct_2021_153 as base build plan,Ox_CTLG_Direct_2021_175 as base build plan,Ox_CTLG_Direct_2021_199 as base build plan,Ox_CTLG_Direct_2021_001 as base build plan,Ox_CTLG_Direct_2021_096 as base build plan,Ox_CTLG_Direct_2021_016 as base build plan
2,2022 Cirrus Benefit Plan ID,,MNS0000290,MNS0000291,MNS0000291,MNS0000292,MNS0000293,MNS0000294,MNS0000295,MNS0000296,...,MNS0000290,MNS0000291,MNS0000291,MNS0000292,MNS0000293,MNS0000294,MNS0000295,MNS0000296,MNS0000297,MNS0000298
3,2022 Cirrus Benefit Bundle Option ID,,5605086/234699026,5605686/2346993876,523756/2335787926,5605086/234699026,5832466/246756956,5605086/234699026,554739/2323467836,523456/123456756,...,5605086/234699026,5605686/2346993876,523756/2335787926,5605086/234699026,5832466/246756956,5605086/234699026,554739/2323467836,523456/123456756,4567823/469902623,56235386/234696901
4,2022 Rx Code,,No-Standard Rx,No-Standard Rx,No-Standard Rx,No-Standard Rx,RX66-100,RX66-100,RX66-100,RX38,...,No-Standard Rx,No-Standard Rx,No-Standard Rx,No-Standard Rx,RX66-100,RX66-100,RX66-100,RX38,RX66-100,RX39-100
5,2023 Comments,,No changes except for row 36,No changes except for row 36,No changes except for row 36,No changes except for row 36,No changes except for row 36,No changes except for row 36,No changes except for row 36,No changes except for row 36,...,No changes except for row 36,No changes except for row 36,No changes except for row 36,No changes except for row 36,No changes except for row 36,No changes except for row 36,No changes except for row 36,No changes except for row 36,No changes except for row 36,No changes except for row 36
6,Plan Type,,Non Standard,Non Standard,Non Standard,Non Standard,Non Standard,Non Standard,Non Standard,Non Standard,...,Non Standard,Non Standard,Non Standard,Non Standard,Non Standard,Non Standard,Non Standard,Non Standard,Non Standard,Non Standard
7,2023 Tracking Number,,Ox_NYLG_Direct_2021_153 as base build plan,Ox_NYLG_Direct_2021_175 as base build plan,Ox_NYLG_Direct_2021_193 as base build plan,Ox_NYLG_Direct_2021_135 as base build plan,Ox_NYLG_Direct_2021_153 as base build plan,Ox_NYLG_Direct_2021_175 as base build plan,Ox_NYLG_Direct_2021_199 as base build plan,Ox_NYLG_Direct_2021_001 as base build plan,...,Ox_CYLG_Direct_2021_153 as base build plan,Ox_CTLG_Direct_2021_175 as base build plan,Ox_CTLG_Direct_2021_193 as base build plan,Ox_CTLG_Direct_2021_135 as base build plan,Ox_CTLG_Direct_2021_153 as base build plan,Ox_CTLG_Direct_2021_175 as base build plan,Ox_CTLG_Direct_2021_199 as base build plan,Ox_CTLG_Direct_2021_001 as base build plan,Ox_CTLG_Direct_2021_096 as base build plan,Ox_CTLG_Direct_2021_016 as base build plan
8,2023 Cirrus Benefit Plan ID,,MNS0000290,MNS0000291,MNS0000291,MNS0000292,MNS0000293,MNS0000294,MNS0000295,MNS0000296,...,MNS0000290,MNS0000291,MNS0000291,MNS0000292,MNS0000293,MNS0000294,MNS0000295,MNS0000296,MNS0000297,MNS0000298
9,2023 Cirrus Benefit Bundle Option ID,,5605086/234699026,5605686/2346993876,523756/2335787926,5605086/234699026,5832466/246756956,5605086/234699026,554739/2323467836,523456/123456756,...,5605086/234699026,5605686/2346993876,523756/2335787926,5605086/234699026,5832466/

In [30]:
# Assuming the first two columns are at index positions 0 and 1
merged_df['Fields'] = merged_df.iloc[:, 1].astype(str) + ' ' + merged_df.iloc[:, 0].astype(str)

# Insert the new column at the start of the DataFrame
merged_df.insert(0, 'Fields', merged_df.pop('Fields'))
merged_df

,Fields,,,,,,,,,,...,,,,,,,,,,
0,Plan Type,Plan Type,,Non Standard,Non Standard,Non Standard,Non Standard,Non Standard,Non Standard,Non Standard,...,Non Standard,Non Standard,Non Standard,Non Standard,Non Standard,Non Standard,Non Standard,Non Standard,Non Standard,Non Standard
1,2022 Tracking Number,2022 Tracking Number,,Ox_NYLG_Direct_2021_153 as base build plan,Ox_NYLG_Direct_2021_175 as base build plan,Ox_NYLG_Direct_2021_193 as base build plan,Ox_NYLG_Direct_2021_135 as base build plan,Ox_NYLG_Direct_2021_153 as base build plan,Ox_NYLG_Direct_2021_175 as base build plan,Ox_NYLG_Direct_2021_199 as base build plan,...,Ox_CTLG_Direct_2021_153 as base build plan,Ox_CTLG_Direct_2021_175 as base build plan,Ox_CTLG_Direct_2021_193 as base build plan,Ox_CTLG_Direct_2021_135 as base build plan,Ox_CTLG_Direct_2021_153 as base build plan,Ox_CTLG_Direct_2021_175 as base build plan,Ox_CTLG_Direct_2021_199 as base build plan,Ox_CTLG_Direct_2021_001 as base build plan,Ox_CTLG_Direct_2021_096 as base build plan,Ox_CTLG_Direct_2021_016 as base build plan
2,2022 Cirrus Benefit Plan ID,2022 Cirrus Benefit Plan ID,,MNS0000290,MNS0000291,MNS0000291,MNS0000292,MNS0000293,MNS0000294,MNS0000295,...,MNS0000290,MNS0000291,MNS0000291,MNS0000292,MNS0000293,MNS0000294,MNS0000295,MNS0000296,MNS0000297,MNS0000298
3,2022 Cirrus Benefit Bundle Option ID,2022 Cirrus Benefit Bundle Option ID,,5605086/234699026,5605686/2346993876,523756/2335787926,5605086/234699026,5832466/246756956,5605086/234699026,554739/2323467836,...,5605086/234699026,5605686/2346993876,523756/2335787926,5605086/234699026,5832466/246756956,5605086/234699026,554739/2323467836,523456/123456756,4567823/469902623,56235386/234696901
4,2022 Rx Code,2022 Rx Code,,No-Standard Rx,No-Standard Rx,No-Standard Rx,No-Standard Rx,RX66-100,RX66-100,RX66-100,...,No-Standard Rx,No-Standard Rx,No-Standard Rx,No-Standard Rx,RX66-100,RX66-100,RX66-100,RX38,RX66-100,RX39-100
5,2023 Comments,2023 Comments,,No changes except for row 36,No changes except for row 36,No changes except for row 36,No changes except for row 36,No changes except for row 36,No changes except for row 36,No changes except for row 36,...,No changes except for row 36,No changes except for row 36,No changes except for row 36,No changes except for row 36,No changes except for row 36,No changes except for row 36,No changes except for row 36,No changes except for row 36,No changes except for row 36,No changes except for row 36
6,Plan Type,Plan Type,,Non Standard,Non Standard,Non Standard,Non Standard,Non Standard,Non Standard,Non Standard,...,Non Standard,Non Standard,Non Standard,Non Standard,Non Standard,Non Standard,Non Standard,Non Standard,Non Standard,Non Standard
7,2023 Tracking Number,2023 Tracking Number,,Ox_NYLG_Direct_2021_153 as base build plan,Ox_NYLG_Direct_2021_175 as base build plan,Ox_NYLG_Direct_2021_193 as base build plan,Ox_NYLG_Direct_2021_135 as base build plan,Ox_NYLG_Direct_2021_153 as base build plan,Ox_NYLG_Direct_2021_175 as base build plan,Ox_NYLG_Direct_2021_199 as base build plan,...,Ox_CYLG_Direct_2021_153 as base build plan,Ox_CTLG_Direct_2021_175 as base build plan,Ox_CTLG_Direct_2021_193 as base build plan,Ox_CTLG_Direct_2021_135 as base build plan,Ox_CTLG_Direct_2021_153 as base build plan,Ox_CTLG_Direct_2021_175 as base build plan,Ox_CTLG_Direct_2021_199 as base build plan,Ox_CTLG_Direct_2021_001 as base build plan,Ox_CTLG_Direct_2021_096 as base build plan,Ox_CTLG_Direct_2021_016 as base build plan
8,2023 Cirrus Benefit Plan ID,2023 Cirrus Benefit Plan ID,,MNS0000290,MNS0000291,MNS0000291,MNS0000292,MNS0000293,MNS0000294,MNS0000295,...,MNS0000290,MNS0000291,MNS0000291,MNS0000292,MNS0000293,MNS0000294,MNS0000295,MNS0000296,MNS0000297,MNS0000298
9,2023 Cirrus Benefit Bundle Option ID,2023 Cirrus Benefit Bundle Option ID,,5605086/234699026,5605686/2346993876,523756/2335787926,5605086/234699026,5832466/246756956,5605086/234699026,554739/2323467836,...,5605086/234699026,5605686/2346993876,523756/2335787926,560

In [31]:
# Remove the second row
merged_df.drop(merged_df.index[1], inplace=True)

In [32]:
# Set the first column as the index
merged_df.set_index(merged_df.columns[0], inplace=True)

# Transpose the DataFrame
transposed_df = merged_df.T

transposed_df

Fields,Plan Type,2022 Cirrus Benefit Plan ID,2022 Cirrus Benefit Bundle Option ID,2022 Rx Code,2023 Comments,Plan Type,2023 Tracking Number,2023 Cirrus Benefit Plan ID,2023 Cirrus Benefit Bundle Option ID,2023 Rx Code,...,IN Outpatient Surgery,OON Outpatient Surgery,IN Diagn Lab and Xray,OON Diagn Lab and Xray,IN Urgent Care,ER Copay Emergency Room,IN Rider DESCRIPTIONS,IN Rider DESCRIPTIONS,IN UCR/MNRP,IN STANDARD CHANGES
,Plan Type,2022 Cirrus Benefit Plan ID,2022 Cirrus Benefit Bundle Option ID,2022 Rx Code,2023 Comments,Plan Type,2023 Tracking Number,2023 Cirrus Benefit Plan ID,2023 Cirrus Benefit Bundle Option ID,2023 Rx Code,...,Outpatient Surgery,Outpatient Surgery,Diagn Lab and Xray,Diagn Lab and Xray,Urgent Care,Emergency Room,Rider DESCRIPTIONS,Rider DESCRIPTIONS,UCR/MNRP,STANDARD CHANGES
,,,,,,,,,,,...,IN,OON,IN,OON,IN,ER Copay,IN,IN,IN,IN
,Non Standard,MNS0000290,5605086/234699026,No-Standard Rx,No changes except for row 36,Non Standard,Ox_NYLG_Direct_2021_153 as base build plan,MNS0000290,5605086/234699026,No-Standard Rx,...,Ded then 20%,Ded then 30%,"Lab: $0, X-ray/Adv imaging: Ded then 10%",Ded then 20%,$40,$500,$20/$200 ded then $45/$85,SAME/OPP SEX DP,200% MNRP,DME is in-network only; Infertility is covered...
,Non Standard,MNS0000291,5605686/2346993876,No-Standard Rx,No changes except for row 36,Non Standard,Ox_NYLG_Direct_2021_175 as base build plan,MNS0000291,5605686/2346993876,No-Standard Rx,...,Ded then 20%,,No Charge,Ded then 20%,$60,$400,$20/$200 ded then $45/$85,SAME/OPP SEX DP,300% MNRP,DME is in-network only; Infertility is covered...
,Non Standard,MNS0000291,523756/2335787926,No-Standard Rx,No changes except for row 36,Non Standard,Ox_NYLG_Direct_2021_193 as base build plan,MNS0000291,523756/2335787926,No-Standard Rx,...,Ded then 20%,Ded then 20%,"Lab: $0, X-ray/Adv imaging: Ded then 10%",Ded then 20%,$40,$500,$20/$200 ded then $45/$85,SAME/OPP SEX DP,400% MNRP,DME is in-network only; Infertility is covered...
,Non Standard,MNS0000292,5605086/234699026,No-Standard Rx,No changes except for row 36,Non Standard,Ox_NYLG_Direct_2021_135 as base build plan,MNS0000292,5605086/234699026,No-Standard Rx,...,$320,,No Charge,Ded then 20%,$30,$700,$20/$200 ded then $45/$85,SAME/OPP SEX DP,100% MNRP,DME is in-network only; Infertility is covered...
,Non Standard,MNS0000293,5832466/246756956,RX66-100,No changes except for row 36,Non Standard,Ox_NYLG_Direct_2021_153 as base build plan,MNS0000293,5832466/246756956,RX66-100,...,Ded then 20%,Ded then 30%,No Charge,Ded then 20%,$40,$500,$20/$200 ded then $45/$85,SAME/OPP SEX DP,200% MNRP,DME is in-network only; Infertility is covered...
,Non Standard,MNS0000294,5605086/234699026,RX66-100,No changes except for row 36,Non Standard,Ox_NYLG_Direct_2021_175 as base build plan,MNS0000294,5605086/234699026,RX66-100,...,Ded then 20%,Ded then 10%,"Lab: $0, X-ray/Adv imaging: Ded then 10%",Ded then 20%,$80,$900,$20/$200 ded then $45/$85,SAME/OPP SEX DP,200% MNRP,DME is in-network only; Infertility is covered...
,Non Standard,MNS0000295,554739/2323467836,RX66-100,No changes except for row 36,Non Standard,Ox_NYLG_Direct_2021_199 as base build plan,MNS0000295,554739/2323467836,RX66-100,...,Ded then 20%,,No Charge,Ded then 20%,#30,$700,$20/$200 ded then $45/$85,SAME/OPP SEX DP,300% MNRP,DME is in-network only; Infertility is covered...
,Non Standard,MNS0000296,523456/123456756,RX38,No changes except for row 36,Non Standard,Ox_NYLG_Direct_2021_001 as base build plan,MNS0000296,523456/123456756,RX38,...,Ded then 20%,Ded then 30%,No Charge,Ded then 20%,$80,$900,$20/$200 ded then $45/$85,SAME/OPP SEX DP,400% MNRP,DME is in-network only; Infertility is covered...


In [33]:
# Delete the first two rows of the transposed DataFrame
transposed_df = transposed_df.iloc[2:]


In [34]:
# Delete the first 5 columns of the transposed DataFrame
transposed_df = transposed_df.iloc[:, 5:]

# Display the modified DataFrame
transposed_df

Fields,Plan Type,2023 Tracking Number,2023 Cirrus Benefit Plan ID,2023 Cirrus Benefit Bundle Option ID,2023 Rx Code,Benefit Plan Descriptions,IN Deductible,OON Deductible,IN Coinsurance,OON Coinsurance,...,IN Outpatient Surgery,OON Outpatient Surgery,IN Diagn Lab and Xray,OON Diagn Lab and Xray,IN Urgent Care,ER Copay Emergency Room,IN Rider DESCRIPTIONS,IN Rider DESCRIPTIONS,IN UCR/MNRP,IN STANDARD CHANGES
,Non Standard,Ox_NYLG_Direct_2021_153 as base build plan,MNS0000290,5605086/234699026,No-Standard Rx,OXF-FRE DIR PPO 6A 2000-90-NY,$2000/$3000,$2000/$5000,90%,60%,...,Ded then 20%,Ded then 30%,"Lab: $0, X-ray/Adv imaging: Ded then 10%",Ded then 20%,$40,$500,$20/$200 ded then $45/$85,SAME/OPP SEX DP,200% MNRP,DME is in-network only; Infertility is covered...
,Non Standard,Ox_NYLG_Direct_2021_175 as base build plan,MNS0000291,5605686/2346993876,No-Standard Rx,OXF-LIB DIR PPO 8b 2000-90-NY,$1000/$3000,$2000/$4000,80%,50%,...,Ded then 20%,,No Charge,Ded then 20%,$60,$400,$20/$200 ded then $45/$85,SAME/OPP SEX DP,300% MNRP,DME is in-network only; Infertility is covered...
,Non Standard,Ox_NYLG_Direct_2021_193 as base build plan,MNS0000291,523756/2335787926,No-Standard Rx,OXF-LIB ACC PPO 16A 2000-100-NY,$2000/$3000,$2000/$5000,90%,60%,...,Ded then 20%,Ded then 20%,"Lab: $0, X-ray/Adv imaging: Ded then 10%",Ded then 20%,$40,$500,$20/$200 ded then $45/$85,SAME/OPP SEX DP,400% MNRP,DME is in-network only; Infertility is covered...
,Non Standard,Ox_NYLG_Direct_2021_135 as base build plan,MNS0000292,5605086/234699026,No-Standard Rx,OXF-FRE DIR PPO 18B 200-60-NY,$2000/$3000,$2000/$5000,80%,60%,...,$320,,No Charge,Ded then 20%,$30,$700,$20/$200 ded then $45/$85,SAME/OPP SEX DP,100% MNRP,DME is in-network only; Infertility is covered...
,Non Standard,Ox_NYLG_Direct_2021_153 as base build plan,MNS0000293,5832466/246756956,RX66-100,OXF-LIB DIR PPO 6A 2000-90-NY,$2000/$3000,$2000/$5000,90%,60%,...,Ded then 20%,Ded then 30%,No Charge,Ded then 20%,$40,$500,$20/$200 ded then $45/$85,SAME/OPP SEX DP,200% MNRP,DME is in-network only; Infertility is covered...
,Non Standard,Ox_NYLG_Direct_2021_175 as base build plan,MNS0000294,5605086/234699026,RX66-100,OXF-FRE PPO 9C 20-100-NY,$2000/$3000,$2000/$5000,70%,60%,...,Ded then 20%,Ded then 10%,"Lab: $0, X-ray/Adv imaging: Ded then 10%",Ded then 20%,$80,$900,$20/$200 ded then $45/$85,SAME/OPP SEX DP,200% MNRP,DME is in-network only; Infertility is covered...
,Non Standard,Ox_NYLG_Direct_2021_199 as base build plan,MNS0000295,554739/2323467836,RX66-100,OXF-LIB EPO 6B 2000-90-NY,$2000/$3000,$2000/$5000,40%,60%,...,Ded then 20%,,No Charge,Ded then 20%,#30,$700,$20/$200 ded then $45/$85,SAME/OPP SEX DP,300% MNRP,DME is in-network only; Infertility is covered...
,Non Standard,Ox_NYLG_Direct_2021_001 as base build plan,MNS0000296,523456/123456756,RX38,OXF-FRE DIR PPO 6A 2000-90-NY,$2000/$3000,$2000/$5000,90%,60%,...,Ded then 20%,Ded then 30%,No Charge,Ded then 20%,$80,$900,$20/$200 ded then $45/$85,SAME/OPP SEX DP,400% MNRP,DME is in-network only; Infertility is covered...
,Non Standard,Ox_NYLG_Direct_2021_096 as base build plan,MNS0000297,4567823/469902623,RX66-100,OXF-FRE PPO 9A 2000-80-NY,$2000/$3000,$2000/$5000,90%,60%,...,Ded then 20%,,No Charge,Ded then 20%,,,$20/$200 ded then $45/$85,SAME/OPP SEX DP,100% MNRP,DME is in-network only; Infertility is covered...
,Non Standard,Ox_NYLG_Direct_2021_016 as base build plan,MNS0000298,56235386/234696901,RX39-100,OXF-LIB DIR PPO 6A 2000-90-NY,$2000/$3000,$2000/$5000,90%,60%,...,Ded then 20%,,"Lab: $0, X-ray/Adv imaging: Ded then 10%",Ded then 20%,$60,$400,$20/$200 ded then $45/$85,SAME/OPP SEX DP,300% MNRP,DME is in-network only; Infertility is covered...


In [35]:
# List of column names you want to remove
columns_to_remove = ['IN Inpatient Hospital', 'OON Inpatient Hospital', 'IN Outpatient Surgery', 'OON Outpatient Surgery', 'IN Diagn Lab and Xray', 'OON Diagn Lab and Xray', 'IN Rider DESCRIPTIONS', 'IN Rider DESCRIPTIONS', 'IN UCR/MNRP', 'IN STANDARD CHANGES']

# Drop the columns
transposed_df = transposed_df.drop(columns=columns_to_remove)

In [36]:
# Assuming your DataFrame is named df
transposed_df.insert(0, 'Group Name', 'ADP')

In [37]:
# Splitting columns except for "2023 Cirrus Benefit Plan ID"
for col in transposed_df.columns:
    if col != '2023 Cirrus Benefit Bundle Option ID' and transposed_df[col].apply(lambda x: '/' in str(x)).any():
        split_cols = transposed_df[col].str.split('/', expand=True)
        transposed_df[f'IND {col}'] = split_cols[0].astype(str).str.strip()
        transposed_df[f'FAMILY {col}'] = split_cols[1].astype(str).str.strip()
        transposed_df.drop(columns=[col], inplace=True)


In [38]:
# Rename columns
transposed_df = transposed_df.rename(columns={
    'IND IN (PCP/SPEC) Office Visit': 'IN PCP Copay',
    'FAMILY IN (PCP/SPEC) Office Visit': 'IN SPEC Copay'
})

transposed_df

Fields,Group Name,Plan Type,2023 Tracking Number,2023 Cirrus Benefit Plan ID,2023 Rx Code,Benefit Plan Descriptions,IN Coinsurance,OON Coinsurance,OON Office Visit,IN Urgent Care,...,IND IN Deductible,FAMILY IN Deductible,IND OON Deductible,FAMILY OON Deductible,IND IN OOP Max - Incl Ded,FAMILY IN OOP Max - Incl Ded,IND OON OOP Max - Incl Ded,FAMILY OON OOP Max - Incl Ded,IN PCP Copay,IN SPEC Copay
,ADP,Non Standard,Ox_NYLG_Direct_2021_153 as base build plan,MNS0000290,No-Standard Rx,OXF-FRE DIR PPO 6A 2000-90-NY,90%,60%,Ded then 20%,$40,...,$2000,$3000,$2000,$5000,$4000,3X,$5000,3X,$35,$60
,ADP,Non Standard,Ox_NYLG_Direct_2021_175 as base build plan,MNS0000291,No-Standard Rx,OXF-LIB DIR PPO 8b 2000-90-NY,80%,50%,Ded then 10%,$60,...,$1000,$3000,$2000,$4000,$4000,2X,$3000,2X,$45,$80
,ADP,Non Standard,Ox_NYLG_Direct_2021_193 as base build plan,MNS0000291,No-Standard Rx,OXF-LIB ACC PPO 16A 2000-100-NY,90%,60%,,$40,...,$2000,$3000,$2000,$5000,$4000,3X,$5000,3X,$35,$60
,ADP,Non Standard,Ox_NYLG_Direct_2021_135 as base build plan,MNS0000292,No-Standard Rx,OXF-FRE DIR PPO 18B 200-60-NY,80%,60%,Ded then 20%,$30,...,$2000,$3000,$2000,$5000,$4000,3X,$5000,3X,$35,$60
,ADP,Non Standard,Ox_NYLG_Direct_2021_153 as base build plan,MNS0000293,RX66-100,OXF-LIB DIR PPO 6A 2000-90-NY,90%,60%,$300,$40,...,$2000,$3000,$2000,$5000,$4000,3X,$5000,3X,$35,$60
,ADP,Non Standard,Ox_NYLG_Direct_2021_175 as base build plan,MNS0000294,RX66-100,OXF-FRE PPO 9C 20-100-NY,70%,60%,Ded then 10%,$80,...,$2000,$3000,$2000,$5000,$4000,3X,$5000,3X,$35,$60
,ADP,Non Standard,Ox_NYLG_Direct_2021_199 as base build plan,MNS0000295,RX66-100,OXF-LIB EPO 6B 2000-90-NY,40%,60%,Ded then 20%,#30,...,$2000,$3000,$2000,$5000,$4000,3X,$5000,3X,$35,$60
,ADP,Non Standard,Ox_NYLG_Direct_2021_001 as base build plan,MNS0000296,RX38,OXF-FRE DIR PPO 6A 2000-90-NY,90%,60%,,$80,...,$2000,$3000,$2000,$5000,$4000,3X,$5000,3X,$35,$60
,ADP,Non Standard,Ox_NYLG_Direct_2021_096 as base build plan,MNS0000297,RX66-100,OXF-FRE PPO 9A 2000-80-NY,90%,60%,Ded then 30%,,...,$2000,$3000,$2000,$5000,$4000,3X,$5000,3X,$35,$60
,ADP,Non Standard,Ox_NYLG_Direct_2021_016 as base build plan,MNS0000298,RX39-100,OXF-LIB DIR PPO 6A 2000-90-NY,90%,60%,Ded then 20%,$60,...,$2000,$3000,$2000,$5000,$4000,3X,$5000,3X,$35,$60


In [39]:
import pandas as pd
import re

def multiply_value(value_str, multiplier):
    try:
        value = int(re.sub(r'[^\d]+', '', value_str))  # Extract numeric part
        result = value * multiplier
        return f"${result}"  # Add a dollar sign before returning the result
    except ValueError:
        return None

# Iterate over each row in the DataFrame
for i in range(len(transposed_df)):
    for col_idx, col in enumerate(transposed_df.columns):
        try:
            if 'X' in str(transposed_df[col].iloc[i]):
                # Get the value from the preceding column in the same row
                value_col = col_idx - 1
                if value_col >= 0 and str(transposed_df.iloc[i, value_col]) != '':
                    value_str = transposed_df.iloc[i, value_col]
                    multiplier_str = transposed_df[col].iloc[i].split('X')[0].strip()
                    if multiplier_str.isdigit():
                        multiplier = int(multiplier_str)
                        # Multiply the preceding value by the multiplier
                        transposed_df[col].iloc[i] = multiply_value(value_str, multiplier)
        except Exception as e:
            print(f"Error: {e}. Skipping row {i} and column {col}")

# Display the updated DataFrame
transposed_df

Fields,Group Name,Plan Type,2023 Tracking Number,2023 Cirrus Benefit Plan ID,2023 Rx Code,Benefit Plan Descriptions,IN Coinsurance,OON Coinsurance,OON Office Visit,IN Urgent Care,...,IND IN Deductible,FAMILY IN Deductible,IND OON Deductible,FAMILY OON Deductible,IND IN OOP Max - Incl Ded,FAMILY IN OOP Max - Incl Ded,IND OON OOP Max - Incl Ded,FAMILY OON OOP Max - Incl Ded,IN PCP Copay,IN SPEC Copay
,ADP,Non Standard,Ox_NYLG_Direct_2021_153 as base build plan,MNS0000290,No-Standard Rx,OXF-FRE DIR PPO 6A 2000-90-NY,90%,60%,Ded then 20%,$40,...,$2000,$3000,$2000,$5000,$4000,$12000,$5000,$15000,$35,$60
,ADP,Non Standard,Ox_NYLG_Direct_2021_175 as base build plan,MNS0000291,No-Standard Rx,OXF-LIB DIR PPO 8b 2000-90-NY,80%,50%,Ded then 10%,$60,...,$1000,$3000,$2000,$4000,$4000,$8000,$3000,$6000,$45,$80
,ADP,Non Standard,Ox_NYLG_Direct_2021_193 as base build plan,MNS0000291,No-Standard Rx,OXF-LIB ACC PPO 16A 2000-100-NY,90%,60%,,$40,...,$2000,$3000,$2000,$5000,$4000,$12000,$5000,$15000,$35,$60
,ADP,Non Standard,Ox_NYLG_Direct_2021_135 as base build plan,MNS0000292,No-Standard Rx,OXF-FRE DIR PPO 18B 200-60-NY,80%,60%,Ded then 20%,$30,...,$2000,$3000,$2000,$5000,$4000,$12000,$5000,$15000,$35,$60
,ADP,Non Standard,Ox_NYLG_Direct_2021_153 as base build plan,MNS0000293,RX66-100,OXF-LIB DIR PPO 6A 2000-90-NY,90%,60%,$300,$40,...,$2000,$3000,$2000,$5000,$4000,$12000,$5000,$15000,$35,$60
,ADP,Non Standard,Ox_NYLG_Direct_2021_175 as base build plan,MNS0000294,RX66-100,OXF-FRE PPO 9C 20-100-NY,70%,60%,Ded then 10%,$80,...,$2000,$3000,$2000,$5000,$4000,$12000,$5000,$15000,$35,$60
,ADP,Non Standard,Ox_NYLG_Direct_2021_199 as base build plan,MNS0000295,RX66-100,OXF-LIB EPO 6B 2000-90-NY,40%,60%,Ded then 20%,#30,...,$2000,$3000,$2000,$5000,$4000,$12000,$5000,$15000,$35,$60
,ADP,Non Standard,Ox_NYLG_Direct_2021_001 as base build plan,MNS0000296,RX38,OXF-FRE DIR PPO 6A 2000-90-NY,90%,60%,,$80,...,$2000,$3000,$2000,$5000,$4000,$12000,$5000,$15000,$35,$60
,ADP,Non Standard,Ox_NYLG_Direct_2021_096 as base build plan,MNS0000297,RX66-100,OXF-FRE PPO 9A 2000-80-NY,90%,60%,Ded then 30%,,...,$2000,$3000,$2000,$5000,$4000,$12000,$5000,$15000,$35,$60
,ADP,Non Standard,Ox_NYLG_Direct_2021_016 as base build plan,MNS0000298,RX39-100,OXF-LIB DIR PPO 6A 2000-90-NY,90%,60%,Ded then 20%,$60,...,$2000,$3000,$2000,$5000,$4000,$12000,$5000,$15000,$35,$60


In [40]:
# Define the column name mappings
column_mappings = {
    'FAMILY OON Deductible': 'FAMILY DEDUCTIBLE OON',
    'IND OON Deductible': 'IND DEDUCTIBLE OON',
    'FAMILY IN Deductible': 'FAMILY DEDUCTIBLE INN',
    'IND IN Deductible': 'IND DEDUCTIBLE INN',
    'FAMILY OON OOP Max - Incl Ded': 'FAMILY OOP OON',
    'IND OON OOP Max - Incl Ded': 'IND OOP OON',
    'FAMILY IN OOP Max - Incl Ded': 'FAMILY OOP INN',
    'IND IN OOP Max - Incl Ded': 'IND OOP INN',
    'IN PCP Copay': 'PCP COPAY',
    'IN Urgent Care': 'URGENT CARE COPAY',
    'ER Copay Emergency Room': 'EMERGENCY ROOM COPAY',
    'IN SPEC Copay': 'SPEC COPAY',
    'IN Coinsurance': 'INN COINSURANCE',
    'OON Coinsurance': 'OON COINSURANCE'
}

# Rename columns
transposed_df = transposed_df.rename(columns=column_mappings)

# Replace empty strings with 'N/A'
transposed_df = transposed_df.replace('', 'N/A')

# Display the DataFrame with renamed columns
transposed_df


Fields,Group Name,Plan Type,2023 Tracking Number,2023 Cirrus Benefit Plan ID,2023 Rx Code,Benefit Plan Descriptions,INN COINSURANCE,OON COINSURANCE,OON Office Visit,URGENT CARE COPAY,...,IND DEDUCTIBLE INN,FAMILY DEDUCTIBLE INN,IND DEDUCTIBLE OON,FAMILY DEDUCTIBLE OON,IND OOP INN,FAMILY OOP INN,IND OOP OON,FAMILY OOP OON,PCP COPAY,SPEC COPAY
,ADP,Non Standard,Ox_NYLG_Direct_2021_153 as base build plan,MNS0000290,No-Standard Rx,OXF-FRE DIR PPO 6A 2000-90-NY,90%,60%,Ded then 20%,$40,...,$2000,$3000,$2000,$5000,$4000,$12000,$5000,$15000,$35,$60
,ADP,Non Standard,Ox_NYLG_Direct_2021_175 as base build plan,MNS0000291,No-Standard Rx,OXF-LIB DIR PPO 8b 2000-90-NY,80%,50%,Ded then 10%,$60,...,$1000,$3000,$2000,$4000,$4000,$8000,$3000,$6000,$45,$80
,ADP,Non Standard,Ox_NYLG_Direct_2021_193 as base build plan,MNS0000291,No-Standard Rx,OXF-LIB ACC PPO 16A 2000-100-NY,90%,60%,N/A,$40,...,$2000,$3000,$2000,$5000,$4000,$12000,$5000,$15000,$35,$60
,ADP,Non Standard,Ox_NYLG_Direct_2021_135 as base build plan,MNS0000292,No-Standard Rx,OXF-FRE DIR PPO 18B 200-60-NY,80%,60%,Ded then 20%,$30,...,$2000,$3000,$2000,$5000,$4000,$12000,$5000,$15000,$35,$60
,ADP,Non Standard,Ox_NYLG_Direct_2021_153 as base build plan,MNS0000293,RX66-100,OXF-LIB DIR PPO 6A 2000-90-NY,90%,60%,$300,$40,...,$2000,$3000,$2000,$5000,$4000,$12000,$5000,$15000,$35,$60
,ADP,Non Standard,Ox_NYLG_Direct_2021_175 as base build plan,MNS0000294,RX66-100,OXF-FRE PPO 9C 20-100-NY,70%,60%,Ded then 10%,$80,...,$2000,$3000,$2000,$5000,$4000,$12000,$5000,$15000,$35,$60
,ADP,Non Standard,Ox_NYLG_Direct_2021_199 as base build plan,MNS0000295,RX66-100,OXF-LIB EPO 6B 2000-90-NY,40%,60%,Ded then 20%,#30,...,$2000,$3000,$2000,$5000,$4000,$12000,$5000,$15000,$35,$60
,ADP,Non Standard,Ox_NYLG_Direct_2021_001 as base build plan,MNS0000296,RX38,OXF-FRE DIR PPO 6A 2000-90-NY,90%,60%,N/A,$80,...,$2000,$3000,$2000,$5000,$4000,$12000,$5000,$15000,$35,$60
,ADP,Non Standard,Ox_NYLG_Direct_2021_096 as base build plan,MNS0000297,RX66-100,OXF-FRE PPO 9A 2000-80-NY,90%,60%,Ded then 30%,N/A,...,$2000,$3000,$2000,$5000,$4000,$12000,$5000,$15000,$35,$60
,ADP,Non Standard,Ox_NYLG_Direct_2021_016 as base build plan,MNS0000298,RX39-100,OXF-LIB DIR PPO 6A 2000-90-NY,90%,60%,Ded then 20%,$60,...,$2000,$3000,$2000,$5000,$4000,$12000,$5000,$15000,$35,$60


In [24]:
# Save the transposed DataFrame to an Excel file
transposed_df.to_excel('temp4_adp.xlsx', index=False)